In [2]:
import pandas as pd
from openai import OpenAI
import json

client = OpenAI()

In [25]:
df = pd.read_xml("data/dataset.xml")
df['prompt'] = df['input']
df['completion'] = df['output']
del df['index']
del df['input']
del df['output']

df.head()

,prompt,completion
0,What is a 'customized integrated framework' in...,In the context of Audit and Assurance policy a...
1,What are some of the elements that audit and a...,Audit and assurance policies and procedures sh...
2,Why is it important to review and update the a...,It's important to review and update the audit ...
3,What is the role of an audit report in the aud...,An audit report plays a crucial role in the au...
4,What is the purpose of follow-up activities in...,Follow-up activities are designed to monitor t...


In [26]:
default_prompt = "{\"messages\": [{\"role\": \"system\", \"content\": \"You are a factual chatbot that is assisting the user with cloud security.\"}"
default_prompt += ", {\"role\": \"user\", \"content\": \""
jsonl_file = ""
for idx, row in df.iterrows():
    jsonl_file += default_prompt + row['prompt'] + "\"}, {\"role\": \"assistant\", \"content\": \"" + row['completion'] + "\"}]}\n"
with open("./data/dataset.jsonl", "w") as file:
    file.write(jsonl_file)

In [27]:
client.files.create(
  
  file=open("./data/dataset.jsonl", "rb"),
  purpose="fine-tune"
)

FileObject(id='file-R98nY1IAiVbZgjwiCMftS52G', bytes=583483, created_at=1718332989, filename='dataset.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [29]:
client.fine_tuning.jobs.create(
  training_file="file-R98nY1IAiVbZgjwiCMftS52G", 
  model="gpt-3.5-turbo-0125"
)

FineTuningJob(id='ftjob-w1IOs1lN93R5BTWaBwBJjWrd', created_at=1718333000, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-YoXe4bdQHnK5IkrEUKFJYxXt', result_files=[], seed=1316398548, status='validating_files', trained_tokens=None, training_file='file-R98nY1IAiVbZgjwiCMftS52G', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)

In [34]:
client.fine_tuning.jobs.retrieve("ftjob-w1IOs1lN93R5BTWaBwBJjWrd")

FineTuningJob(id='ftjob-w1IOs1lN93R5BTWaBwBJjWrd', created_at=1718333000, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-3.5-turbo-0125:personal::9ZrzIuPt', finished_at=1718336035, hyperparameters=Hyperparameters(n_epochs=3, batch_size=2, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-YoXe4bdQHnK5IkrEUKFJYxXt', result_files=['file-OGpKkRglNfJez7bHlQv3hqV3'], seed=1316398548, status='succeeded', trained_tokens=284766, training_file='file-R98nY1IAiVbZgjwiCMftS52G', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)

In [18]:
completion = client.chat.completions.create(
  model="ft:gpt-3.5-turbo-0125:personal::9ZrzIuPt",
  messages=[
    {"role": "system", "content": "Hello!"},
    {"role": "user", "content": "What does the high-risk environment refer to in network architecture documentation?"},
    {"role": "system", "content": "A high-risk environment refers to an IT environment where there is a greater probability and potential for security incidents. This could be due to various factors such as the type of data the system handles, the number of users accessing the system, the use of public networks, or the potential for physical access to sensitive locations."},
    {"role": "user", "content": "What do you mean by \'the use of public networks\'"}
  ]
)

In [19]:
completion.choices[0].message.content

"The use of public networks implies that some elements of the system's network may rely on external, uncontrolled networks for connectivity. This can increase the exposure to potential security threats."